<a href="https://colab.research.google.com/github/ElenaShargina/patterns/blob/master/%D0%9F%D0%BE%D1%80%D0%BE%D0%B6%D0%B4%D0%B0%D1%8E%D1%89%D0%B8%D0%B5%20%D0%BF%D0%B0%D1%82%D1%82%D0%B5%D1%80%D0%BD%D1%8B/Abstract_factory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstract factory / Абстрактная фабрика
Предоставляет интерфейс для содания семейств взаимосвязанных объектов, не специфицируя их конкретных классов.

## Абстрактный "интерфейс" продукта фабрики и абстрактная фабрика
Продуктом нашей фабрики будет Лабиринт. 

In [63]:
from abc import ABC
from random import Random
"""
абстрактный интерфейс для продукта фабрики - лабиринта
"""
class AbstractMaze(ABC):
    def draw(self):
        pass
    def add_room(self, r):
        pass
    def add_door(self,r1,r2):
        pass
"""
абстрактная фабрика для изготовления лабиринтов вне зависимости от их реализации,
главное, чтобы их продукт удовлетворял интерфейсу абстрактного продукта
"""
class MazeFactory (ABC):
    def make_maze(self):
        pass
    def make_room(self):
        pass

## Класс - пользователь фабрики

In [64]:
"""
класс, использующий абстрактную фабрику лабиринтов. На вход нужно дать только указание о том,
какую конкретную реализацию лабиринтов использовать. Создается лабиринт из 4 комнат и двух дверей между ними.
"""
class MazeGame:
    def create_maze(self,maze_factory:MazeFactory):
        m = maze_factory.make_maze()
        r1 = maze_factory.make_room()
        r2 = maze_factory.make_room()
        r3 = maze_factory.make_room()
        r4 = maze_factory.make_room()
        m.add_room(r1)
        m.add_room(r2)
        m.add_room(r3)
        m.add_room(r4)

        m.add_door(r1,r2)
        m.add_door(r2,r3)

        print(m.draw())
        return m


## Цифровой лабиринт - первая конкретная фабрика и конкретный продукт

In [65]:
"""
класс конкретной фабрики, реализующей интерфейс абстрактной
"""
class NumberMazeFactory(MazeFactory):
    def make_maze(self):
        return NumberMaze()
    def make_room(self):
        return NumberRoom()

"""
класс конкретного продукта NumberMaze соответствует интерфейсу абстрактного продукта
"""
class NumberMaze(AbstractMaze):
    _data:list
    def __init__(self):
        self._data = list()

    def draw(self):
        res = ''
        for i in self._data:
            if type(i)==NumberRoom:
               res += ' '+str(i.index)+' '
            else:
                res += str(i)
        return res

    def add_room(self, r):
        self._data.append(r)

    def add_door(self,r1,r2):
        index_r1 = self._data.index(r1)
        index_r2 = self._data.index(r2)
        self._data = self._data[:index_r1+1]+['->']+self._data[index_r2:]

class NumberRoom():
    all_indexes = []
    index:int
    def __init__(self):
        new_index = 0
        while new_index in NumberRoom.all_indexes or new_index==0:
            new_index = Random().randint(1,10**6)
        self.index = new_index

## Графический лабиринт - вторая конкретная фабрика и конкретный продукт

In [66]:
"""
класс конкретной фабрики, реализующей интерфейс абстрактной
"""
class SimpleMazeFactory(MazeFactory):
    def make_maze(self):
        return SimpleMaze()
    def make_room(self):
        return SimpleRoom()
"""
класс конкретного продукта, реализующий интерфейс абстрактного
(Maze наследуем для удобства, чтобы не переносить сюда весь код)
"""
class SimpleMaze(Maze,AbstractMaze):
    pass
class SimpleRoom(Room):
    pass

In [67]:
from enum import Enum
class DrawException(Exception):
    pass

class Direction(Enum):
    north='N'
    south='S'
    east='E'
    west = 'W'

class Incline(Enum):
    vertical = 'V'
    horizontal = 'H'

class MapSite:
    width=5
    vertical_symbol = '|'
    horizontal_symbol = '--'
    corner_symbol = '.'
    empty_symbol = '  '
    door_symbol = '+'
    def enter(self):
        pass
    def draw(self):
        pass

class Room(MapSite):
    all_numbers: list=[]
    def __init__(self):
        new_index = 0
        while new_index in Room.all_numbers or new_index==0:
            new_index = Random().randint(1,10**6)
        Room.all_numbers.append(new_index)
        self.number = new_index
        self.sides = {}
        for i in Direction:
            self.set_side(Wall(),i)

    number: int = None
    sides: dict

    def set_side(self,side,direction:Direction):
        if side.incline == None:
            if direction in (direction.north, direction.south):
                side.incline = Incline.horizontal
            elif direction in (direction.east, direction.west):
                side.incline = Incline.vertical
        self.sides[direction.value] = side

    def set_door(self,direction:Direction):
        self.set_side(Door(),direction)

    def draw(self):
        """
        .------.
        |      |
        |      |
        |      |
        .------.
        """
        res = self.corner_symbol
        if self.sides[Direction.north.value]:
            res += self.sides[Direction.north.value].draw()
        else:
            res += Wall(incline=Incline.horizontal).draw()
        res += self.corner_symbol+'\n'
        vert_res = ''
        if self.sides[Direction.west.value]:
            vert_res += self.sides[Direction.west.value].draw()
        else:
            vert_res += Wall(incline=Incline.vertical).draw()
        if self.sides[Direction.east.value]:
            vert_res += self.sides[Direction.east.value].draw()
        else:
            vert_res += Wall(incline=Incline.vertical).draw()
        vert_res = vert_res.strip().split('\n')
        for i in range(0,len(vert_res)//2):
            res += vert_res[i]+self.empty_symbol*self.width+vert_res[len(vert_res)//2+i]+'\n'
        res += self.corner_symbol
        if self.sides[Direction.south.value]:
            res += self.sides[Direction.south.value].draw()
        else:
            res += Wall(incline=Incline.horizontal).draw()
        res += self.corner_symbol+'\n'
        return res

class Wall(MapSite):
    incline:Incline = None
    def __init__(self, incline=None):
        self.incline = incline

    def draw(self):
        if self.incline == Incline.vertical:
            return f'{self.vertical_symbol}\n'*self.width
        elif self.incline == Incline.horizontal:
            return self.horizontal_symbol*self.width
        else:
            raise DrawException(f'Can not draw wall {self}, no incline is specified.')

class Door(Wall):
    is_open: bool = False
    # room_1: Room = None
    # room_2: Room = None

    def draw(self):
        if self.incline == Incline.vertical:
            wall_height = self.width//3
            door_height = self.width- 2*wall_height
            return f'{self.vertical_symbol}\n'*wall_height+f'{self.door_symbol}\n'*door_height+f'{self.vertical_symbol}\n'*wall_height
        elif self.incline == Incline.horizontal:
            wall_width = self.width // 3
            door_width = self.width - 2 * wall_width
            return self.vertical_symbol * wall_width + self.door_symbol * door_width + self.vertical_symbol * wall_width
        else:
            raise DrawException(f'Can not draw door {self}, no incline is specified.')

class Maze:
    data: list = []
    def add_room(self, room: dict(type=Room,help='help string'))->None:
        self.data.append(room)

    def add_door(self,room1:Room,room2:Room):
        room1.set_door(Direction.east)
        room2.set_door(Direction.west)

    def draw(self):
        res = ''
        res_list = []
        number_of_rooms = len(self.data)
        for r in self.data:
            res_list.append(list(r.draw().splitlines()))
        i = 0
        room_height = len(res_list[0])
        while i<room_height:
            for r in res_list:
                res += r[i]
            res += '\n'
            i+=1
        return res

## Применение

In [70]:
  # Создаем цифровой лабиринт
me = MazeGame().create_maze(NumberMazeFactory())
# me.draw()

 12778 -> 360079 -> 84257  580755 


In [71]:
# Создаем графический лабиринт
ms = MazeGame().create_maze(SimpleMazeFactory())
# ms.draw()

.----------..----------..----------..----------..----------..----------..----------..----------.
|          ||          ||          ||          ||          ||          ||          ||          |
|          ++          ++          ||          ||          ++          ++          ||          |
|          ++          ++          ||          ||          ++          ++          ||          |
|          ++          ++          ||          ||          ++          ++          ||          |
|          ||          ||          ||          ||          ||          ||          ||          |
.----------..----------..----------..----------..----------..----------..----------..----------.

